## Setup file path manager

In [1]:
from data_preparation import filepaths

path_manager = filepaths.PathManager("./data")


## Download audio files and metadata from xeno-canto

In [ ]:
from data_preparation import downloader

downloader = downloader.XenoCantoDownloader(path_manager)

downloader.create_datasets(
    ["Petronia petronia"], min_quality="A", sound_types=["song"], maximum_number_of_background_species=1)

## Create spectrograms from audio files

In [ ]:
from data_preparation import spectrograms

spectrogram_creator = spectrograms.SpectrogramCreator(
    1000, path_manager)

spectrogram_creator.create_spectrograms_for_datasets(datasets=["train", "val", "test"])

## Create datasets and dataloaders

In [2]:
from training import dataset
from torch.utils.data import DataLoader

train_set = dataset.XenoCantoSpectrograms(
    path_manager, chunk_length=1000, split="train")
train_loader = DataLoader(
    train_set, batch_size=2, shuffle=True, num_workers=0)


val_set = dataset.XenoCantoSpectrograms(
    path_manager, chunk_length=1000, split="val")
val_loader = DataLoader(
    val_set, batch_size=2, shuffle=True, num_workers=0)

test_set = dataset.XenoCantoSpectrograms(
    path_manager, chunk_length=1000, split="test")
test_loader = DataLoader(
    test_set, batch_size=2, shuffle=True, num_workers=0)

True


## Train model

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import copy
from torchvision import models

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('----------')

        # Training Phase
        model.train()
        running_loss = 0.0
        running_corrects = 0

        for images_train, labels_train in train_loader:
            # TODO: Labels muessen hier noch richtig als Tensor übergeben werden
            labels_train = torch.zeros(images_train.shape[0], dtype=torch.long)
            images_train = images_train.to(device)
            labels_train = labels_train.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(True):
                images_out = model(images_train)
                _, preds = torch.max(images_out, 1)
                loss = criterion(images_out, labels_train)

                loss.backward()
                optimizer.step()

            running_loss += loss.item() * images_train.size(0)
            running_corrects += torch.sum(preds == labels_train.data)

            scheduler.step()

            epoch_loss = running_loss / len(train_set)
            epoch_acc = running_corrects.double() / len(train_set)

           # print('{} Loss: {:.4f} Acc: {:.4f}'.format(
           #     'train', epoch_loss, epoch_acc))

            # Evaluate Model
            model.eval()
            running_loss = 0.0
            running_corrects = 0

            for images_val, labels_val in val_loader:
                # TODO: Labels muessen hier noch richtig als Tensor übergeben werden
                labels_val = torch.zeros(images_val.shape[0], dtype=torch.long)
                images_val = images_val.to(device)
                labels_val = labels_val.to(device)

                optimizer.zero_grad()

                outputs = model(images_val)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels_val)

                running_loss += loss.item() * images_val.size(0)
                running_corrects += torch.sum(preds == labels_val.data)

            epoch_loss = running_loss / len(val_set)
            epoch_acc = running_corrects.double() / len(val_set)

           # print('{} Loss: {:.4f} Acc: {:.4f}'.format(
            #    'val', epoch_loss, epoch_acc))

            if epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    return model

resnet_model = models.resnet18()
resnet_model.to(device)

my_criterion = nn.CrossEntropyLoss()
my_optimizer = optim.SGD(resnet_model.parameters(), lr=0.001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(my_optimizer, step_size=7, gamma=0.1)

my_model = train_model(resnet_model, my_criterion, my_optimizer, exp_lr_scheduler,
                       num_epochs=25)

cuda:0
Epoch 0/24
----------

Epoch 1/24
----------

Epoch 2/24
----------

Epoch 3/24
----------

Epoch 4/24
----------

Epoch 5/24
----------

Epoch 6/24
----------

Epoch 7/24
----------

Epoch 8/24
----------

Epoch 9/24
----------

Epoch 10/24
----------

Epoch 11/24
----------

Epoch 12/24
----------

Epoch 13/24
----------

Epoch 14/24
----------

Epoch 15/24
----------

Epoch 16/24
----------

Epoch 17/24
----------

Epoch 18/24
----------

Epoch 19/24
----------

Epoch 20/24
----------

Epoch 21/24
----------

Epoch 22/24
----------

Epoch 23/24
----------

Epoch 24/24
----------

Training complete in 13m 17s
Best val Acc: 1.000000
